In [ ]:
from turtle import color
import numpy as np
import cv2
import argparse
from matplotlib import pyplot as plt
import os

画像を回転させる

In [ ]:
MIN_MATCH_COUNT = 2

parser = argparse.ArgumentParser(description='Template matcher')
parser.add_argument('--template', type=str, action='store',
                    help='The image to be used as template')
parser.add_argument('--map', type=str, action='store',
                    help='The image to be searched in')
parser.add_argument('--show', action='store_true',
                    help='Shows result image')
parser.add_argument('--save-dir', type=str, default='./',
                    help='Directory in which you desire to save the result image')

args = parser.parse_args()


def get_matched_coordinates(temp_img, map_img):
    """
    Gets template and map image and returns matched coordinates in map image
    Parameters
    ----------
    temp_img: image
        image to be used as template
    map_img: image 
        image to be searched in
    Returns
    ---------
    ndarray
        an array that contains matched coordinates
    """

    # initiate SIFT detector
    sift = cv2.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(temp_img, None)
    kp2, des2 = sift.detectAndCompute(map_img, None)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # find matches by knn which calculates point distance in 128 dim
    matches = flann.knnMatch(des1, des2, k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m, n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    if len(good) > MIN_MATCH_COUNT:
        src_pts = np.float32(
            [kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32(
            [kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

        # find homography
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        matchesMask = mask.ravel().tolist()

        h, w = temp_img.shape
        pts = np.float32([[0, 0], [0, h-1], [w-1, h-1],
                          [w-1, 0]]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, M)  # matched coordinates
        
        map_img = cv2.polylines(
            map_img, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

    else:
        print("Not enough matches are found - %d/%d" %
              (len(good), MIN_MATCH_COUNT))
        matchesMask = None

    draw_params = dict(matchColor=(0, 255, 0),  # draw matches in green color
                       singlePointColor=None,
                       matchesMask=matchesMask,  # draw only inliers
                       flags=2)

    # draw template and map image, matches, and keypoints
    img3 = cv2.drawMatches(temp_img, kp1, map_img, kp2,
                           good, None, **draw_params)

    # if --show argument used, then show result image
    if args.show:
        plt.imshow(img3, 'gray'), plt.show()

    # result image path
    if(args.save_dir != ''):
        cv2.imwrite(os.path.join(args.save_dir), img3)
    else:
        cv2.imwrite(os.path.join(args.save_dir, 'output.jpg'), img3)

    return dst


if __name__ == "__main__":

    # read images
    temp_img_gray = cv2.imread(args.template, 0)
    map_img_gray = cv2.imread(args.map, 0)

    # equalize histograms
    temp_img_eq = cv2.equalizeHist(temp_img_gray)
    map_img_eq = cv2.equalizeHist(map_img_gray)

    # calculate matched coordinates
    coords = get_matched_coordinates(temp_img_eq, map_img_eq)

    print(coords)

画像を合成する

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Stitch given images.')
    parser.add_argument("inputs", metavar='input', type=str, nargs='+', help='input file')
    parser.add_argument("--output", type=str, default='a.png', help='output file (default=a.png)')
    args = parser.parse_args()

    input_images = []
    for i in args.inputs:
        image = cv2.imread(i)
        if image is None:
            print(f'Error: Unable to open file "{i}".')
            exit()
        input_images.append(image)

    if len(input_images) == 1:
        cv2.imwrite(args.output, input_images[0])
    else:
        stitcher = cv2.createStitcher(True)
        stitched = stitcher.stitch(input_images)
        cv2.imwrite(args.output, stitched[1])